### Load the .env variables

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

api_key = os.environ.get("OPENAI_API_KEY")

if not api_key:
    raise ValueError("OPENAI_API_KEY is not set in the enviroment variables")

### Agents

Agents in essense are LLMs that accomplish a specific task. They can be supplemented with **tools**, **structured output**, and **handoff** to other agents.

In [11]:
# import Agent and Runner and define the llm model
from agents import Agent, Runner

from pydantic import BaseModel

llm_model = "gpt-4o-mini"

#### Create agents

In [ ]:
# a basic agent
basic_agent = Agent(
    name = "Basic Agent",
    instructions = "You are a helpful assistant. Repond on in all caps.",
    model = llm_model
)

result = await Runner.run(basic_agent, "Hello!, How are you?")
result.final_output

'HELLO! I’M DOING GREAT, THANK YOU! HOW ABOUT YOU?'

In [8]:
# another agent
joke_agent = Agent(
    name = "Joke Agent",
    instructions = "You are a joke teller. You are given a topic and you need to tell a joke about it.",
    model = llm_model
)

topic = "AI"

result = await Runner.run(joke_agent, topic)
result.final_output

'Why did the AI go to therapy?\n\nBecause it had too many unresolved algorithms!'

In [10]:
# a new agent working with an existing agent
language_agent = Agent(
    name = "Language Agent",
    instructions = "You are a language expert. You are given a joke and you need to rewrite it in a different language.",
    model = llm_model
)

joke_result = await Runner.run(joke_agent, topic)
translated_result = await Runner.run(language_agent, f"Translate this joke to Spanish: {joke_result.final_output}")

print(f"Original joke:\n{joke_result.final_output}\n")
print(f"Translated joke:\n{translated_result.final_output}\n")

Original joke:
Why did the AI go broke?

Because it couldn't find the right algorithm for making cents!

Translated joke:
¿Por qué se arruinó la IA?

¡Porque no pudo encontrar el algoritmo adecuado para hacer centavos!



#### Structured outputs

Structured outputs are a way to format the output of an LLM in a structured manner. This can be useful for task that require specific formatting or data extraction.

In [12]:
class Recipe(BaseModel):
    title: str
    ingredients: list[str]
    cooking_time: int # in minutes
    servings: int

recipe_agent = Agent(
    name = "Recipe Agent",
    instructions = ("You are an agent for creating recipes. You will be given the name of a food and your job"
                    " is to output that as an actual detailed recipe. The cooking time should be in minutes."),
    output_type = Recipe
)

response = await Runner.run(recipe_agent, "Italian Sasuage with Spaghetti")
response.final_output

Recipe(title='Italian Sausage with Spaghetti', ingredients=['400g (14 oz) spaghetti', '4 Italian sausages (mild or hot, as preferred)', '2 tablespoons olive oil', '1 onion, finely chopped', '3 garlic cloves, minced', '1 can (400g/14 oz) crushed tomatoes', '2 tablespoons tomato paste', '1 teaspoon dried oregano', '1 teaspoon dried basil', '1/2 teaspoon chili flakes (optional)', 'Salt and pepper to taste', 'Parmesan cheese, grated (for serving)', 'Fresh basil leaves (for garnish)'], cooking_time=40, servings=4)